# Generating metadata

### What this notebook does
**Step 1:** Create metadata file, containing a bunch of JSON-formatted trial metadata objects

**Step 2:**  Insert each trial as a record into a mongo database

This assumes that the stimuli have been uploaded to the S3 bucket using `upload_stims_to_s3.ipynb`.

In [ ]:
#Which experiment? bucket_name is the name of the experiment and will be name of the databases both on mongoDB and S3
bucket_name = 'human-physics-benchmarking-XXX-pilot' #CHANGE THIS ⚡️
stim_version = 'production_1' #CHANGE THIS ⚡️
random_seed = 42

In [ ]:
import os
import numpy as np
from PIL import Image
import pandas as pd
import json
import pymongo as pm
from glob import glob
from IPython.display import clear_output
import ast
import itertools
import random
import h5py

In [ ]:
#set random number state
rng = np.random.RandomState(seed=random_seed)

In [ ]:
def list_files(paths, ext='mp4'):
    """Pass list of folders if there are stimuli in multiple folders. 
    Make sure that the containing folder is informative, as the rest of the path is ignored in naming. 
    Also returns filenames as uploaded to S3"""
    if type(paths) is not list:
        paths = [paths]
    results = []
    names = []
    for path in paths:
        results += [y for x in os.walk(path) for y in glob(os.path.join(x[0], '*.%s' % ext))]
        names += [os.path.basename(os.path.dirname(y))+'_'+os.path.split(y)[1] for x in os.walk(path) for y in glob(os.path.join(x[0], '*.%s' % ext))]
    hdf5s = [r.split("_img.")[0]+".hdf5" for r in results]
    return results,names,hdf5s

In [ ]:
## helper to build stim urls
def build_s3_url(filename, bucket_name):    
    return 'https://{}.s3.amazonaws.com/{}'.format(bucket_name, filename)

In [ ]:
local_stem = 'XXX' #CHANGE THIS ⚡️ # needs trailing /
dirnames = [d.split('/')[-1] for d in glob(local_stem+'/*')]
data_dirs = [local_stem + d for d in dirnames]

dataset_name = '{}_{}'.format(bucket_name, stim_version)
stimulus_extension = "mp4" #what's the file extension for the stims? Provide without dot

## get a list of paths to each one
full_stim_paths,filenames, _  = list_files(data_dirs,stimulus_extension)
full_map_paths, mapnames, _ = list_files(data_dirs, ext = 'png') #generate filenames and stimpaths for target/zone map
full_hdf5_paths,hdf5_names, _ = list_files(data_dirs,ext = 'hdf5')
print('We have {} stimuli to evaluate.'.format(len(full_stim_paths)))

In [ ]:
assert all('_img' in p for p in full_stim_paths), "Make sure to only pass in `_img` passes!"

In [ ]:
## basic metadata lists
stim_urls = [build_s3_url(p,bucket_name) for p in filenames]
stim_map_urls = [build_s3_url(p,bucket_name) for p in mapnames]
stim_IDs = [name.split('.')[0] for name in filenames]
stim_hdf5_urls = [build_s3_url(p,bucket_name) for p in hdf5_names]
hdf5_paths = ['/'.join(p.split('/')[-2:]) for p in full_hdf5_paths]
stim_sets = [p.split('/')[-2:-1][0] for p in full_stim_paths]

In [ ]:
## convert to pandas dataframe
M = pd.DataFrame([stim_IDs,stim_urls, stim_map_urls, hdf5_paths, stim_hdf5_urls, stim_sets]).transpose()
M.columns = ['stim_ID', 'stim_url', 'map_url', 'hdf5_path','hdf5_url', 'stim_set']

Add metadata to the stimuli

From `metadata.json`:

In [ ]:
# if needed, add code to add additional columns
# Add trial labels to the metadata using the stimulus metadata.json
target_hit_zone_labels = dict()
for _dir in data_dirs:
    with open(_dir + '/metadata.json', 'rb') as f:
        trial_metas = json.load(f)
        
    for i,meta in enumerate(trial_metas):
        stim_name = meta['stimulus_name']
        if stim_name == 'None': #recreate stimname from order in metadata
            stim_name = str(i).zfill(4)
            stim_name = _dir.split('/')[-1] + '_' + stim_name
        if stim_name[-4:] != "_img": stim_name+='_img' #stimnames need to end in "_img"
        label = meta['does_target_contact_zone']
        target_hit_zone_labels[stim_name] = label
        
print("num positive labels: %d" % sum(list(target_hit_zone_labels.values())))
print("num negative labels: %d" % (len(target_hit_zone_labels) - sum(list(target_hit_zone_labels.values()))))

In [ ]:
# make new df with all metadata
GT = pd.DataFrame([list(target_hit_zone_labels.keys()), list(target_hit_zone_labels.values())]).transpose()
GT.columns = ['stim_ID', 'target_hit_zone_label']

In [ ]:
#sanity check—this should be empty save for maybe leftover familiarization stims
set(M['stim_ID']).symmetric_difference(GT['stim_ID'])

In [ ]:
# merge with M
# if all([col not in M.columns for col in GT.columns[1:]]):
M = M.merge(GT, on='stim_ID')
print("added labels %s" % list(GT.columns[1:]))

We also want to have a number of familiarization trials. The stims are expected in the S3 bucket with that filename. 

In [ ]:
familiarization_stem = 'XXX' #CHANGE THIS ⚡️

NUM_FAM_TRIALS = 5 #how many familiarizationt trials per condition?

In [ ]:
#sample an equal amount of positive and negative trials from the familiarization stem
positive_rows_fam = M[(M['target_hit_zone_label'] == True) & (M['stim_set'] == familiarization_stem)].to_dict(orient='records')
positive_rows_fam = list(rng.choice(positive_rows_fam,NUM_FAM_TRIALS,replace=False))
negative_rows_fam = M[(M['target_hit_zone_label'] == False) & (M['stim_set'] == familiarization_stem)].to_dict(orient='records')
negative_rows_fam = list(rng.choice(negative_rows_fam,NUM_FAM_TRIALS,replace=False))
all_rows_fam = positive_rows_fam + negative_rows_fam
rng.shuffle(all_rows_fam)

In [ ]:
familiarization_M = pd.DataFrame(all_rows_fam)
# save the familiariaziation dict
familiarization_trials = familiarization_M.transpose().to_dict()
# needs to have strings as keys
familiarization_trials = {str(key):value for key, value in familiarization_trials.items()}

In [ ]:
familiarization_M

In [ ]:
assert len(familiarization_M) == len(familiarization_M['stim_ID'].unique())

In [ ]:
len(M),len(familiarization_M)

Get a set of main stimuli

In [ ]:
## drop the stims that have the same stem as the familiarization trials
M = M[~(M['stim_set'] == familiarization_stem)]

In [ ]:
len(M),len(familiarization_M)

In [ ]:
# remove some bad stimuli -- regenerate these
bad_stimuli = [
    #filename of bad stimuli #CHANGE THIS IF NEEDED ⚡️
]

for nm in bad_stimuli:
    M = M[~M['stim_ID'].str.contains(nm)]

In [ ]:
len(M)

We want to make sure that we have the same amount of stimuli. To prevent data from being too sparse, we also might want to sample a subset of stimuli.

In [ ]:
STIM_SET_SIZE = 150 #how many total stimuli do we want?

In [ ]:
per_label_num = min(len(M[M['target_hit_zone_label'] == False]),
                   len(M[M['target_hit_zone_label'] == True]),
                   int(STIM_SET_SIZE/2))
print("We get", per_label_num, "stimuli per label for a total of", per_label_num * 2)

In [ ]:
positive_rows = M[M['target_hit_zone_label'] == True].to_dict(orient='records')
positive_rows = list(rng.choice(positive_rows,per_label_num,replace=False))
negative_rows = M[M['target_hit_zone_label'] == False].to_dict(orient='records')
negative_rows = list(rng.choice(negative_rows,per_label_num,replace=False))
all_rows = positive_rows + negative_rows

In [ ]:
# recreate M
M = pd.DataFrame(all_rows)

In [ ]:
M

In [ ]:
assert len(M) == len(M['stim_ID'].unique())

In [ ]:
#lets save the experimental stims out into a json
M.to_json(bucket_name+'_'+stim_version+'_experimental_stims'+".json")

From `hdf5s` get all the metadata:

In [ ]:
metadata = {} #holds all the metadata for all stimuli

for name,hdf5_path in zip([f.split('.')[0] for f in filenames],full_hdf5_paths):
    #load hdf5
#     print("loading",hdf5_path)
    try:
        hdf5 = h5py.File(hdf5_path,'r') #get the static part of the HDF5
        stim_name = str(np.array(hdf5['static']['stimulus_name']))
        metadatum = {} #metadata for the current stimulus
        for key in hdf5['static'].keys():
            datum = np.array(hdf5['static'][key])
            if datum.shape == (): datum = datum.item() #unwrap non-arrays
            metadatum[key] = datum
        #close file
        hdf5.close()
        metadata[name] = metadatum
    except Exception as e:
        print("Error with",hdf5_path,":",e)
        continue


In [ ]:
metadata[list(metadata.keys())[0]]

Insert those metadatas into M:

In [ ]:
for index in M.index:
    stim_name = M.at[index,'stim_ID']
    for key,value in metadata[stim_name].items():
        M.at[index,key] = str(value) #insert every item as string

Ok, so we now have a list of all the stimuli to evaluate. Let's create random orders of them and submit them to the database. Set the number of stimuli shown to one participant and the number of different orders to generate (`num_of_permutations`). Each of these sets can be shown to a participant, with the least often shown set being shown (so we can get away with fewer sets than subjects). 

In [ ]:
M

In [ ]:
len(M)

In [ ]:
stims_per_run = len(M) #len(M) to show all stimuli
num_of_sets = 125 #how many different orders to produce?

In [ ]:
#generate list of stimuli as dicts
L = M.to_dict(orient='records')

In [ ]:
#generate list of lists of shuffled, sampled w/o replacement stimuli
stim_sets = []
for i in range(num_of_sets):
    stim_set = list(rng.choice(L, stims_per_run, replace=False))
    stim_sets += [{str(num):stim for num,stim in enumerate(stim_set)}]

In [ ]:
len(stim_sets)

In [ ]:
familiarization_M

In [ ]:
#these will be our familiarization trials
familiarization_trials = familiarization_M.transpose().to_dict()
familiarization_trials = {str(key):value for key, value in familiarization_trials.items()}
# familiarization_trials

Now we have a list of sequences of stimuli.

Set up ssh bridge to write to mongodb. Insert your username. If you don't have an SSH secret set yet, run `ssh -fNL 27017:127.0.0.1:27017 USERNAME@cogtoolslab.org` in your shell. \
*CHANGE THIS ⚡️*

In [ ]:
!ssh -fNL 27017:127.0.0.1:27017 XXX@cogtoolslab.org

In [ ]:
# set vars 
auth = pd.read_csv('../auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user. Place it in the toplevel of the repo
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org' ## cogtoolslab ip address

conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['stimuli']
coll = db[dataset_name]

In [ ]:
print(dataset_name)

In [ ]:
#get list of current collections
sorted(db.list_collection_names())

In [ ]:
#⚠️drop collection if necessary. 
db.drop_collection(dataset_name) 

Let's **do it**!

In [ ]:
#lets save the experiment structure out into a json as well
#rows are different games, columns are the full metadata for the nth stimulus
pd.DataFrame(stim_sets).to_json(bucket_name+'_'+stim_version+'_experiment'+".json")

In [ ]:
## actually add data now to the database
for (i,m) in enumerate(stim_sets):
    coll.insert_one({'stims':m, 'familiarization_stims': familiarization_trials})
    print('{} of {}'.format(i+1, len(stim_sets)))
    clear_output(wait=True)

print('Done inserting records into mongo! The collection name is',dataset_name)

In [ ]:
coll.estimated_document_count()

Let's pull a bit from the database to check if it worked

In [ ]:
coll.find_one()

In [ ]:
list(coll.find())